In [1]:
import numpy as np

with open('input.txt', 'r') as file:
    data = file.read()
    data = data.split('\n')
    
count_tasks = int(data[0])
difficult = np.array(list(map(int, data[1].split())))
timeh = np.array(list(map(float, data[2].split())))
count_empoyee = int(data[3])
params = np.array([list(map(float, i.split())) for i in data[4:]])
    
print(count_tasks, 
      difficult,
      timeh, 
      count_empoyee,
      params,
      sep='\n')

#print(np.random.randint(low=1, high=count_empoyee+1, size=[2, count_tasks], dtype=int))

1000
[2 3 1 2 4 4 2 2 2 4 3 3 4 4 1 3 2 4 4 1 3 1 2 2 3 2 3 2 4 4 4 3 2 3 4 4 3
 1 3 4 1 1 2 2 3 2 1 3 4 3 2 2 4 2 2 1 2 3 2 3 2 4 3 2 3 1 3 2 1 4 2 4 4 4
 3 2 2 1 4 1 2 4 2 1 4 1 2 2 4 3 2 3 3 1 1 3 2 1 4 4 4 4 1 2 2 1 4 1 3 2 3
 1 4 4 3 3 1 1 4 1 4 1 1 2 2 4 1 3 3 3 3 4 1 4 2 4 2 1 1 2 1 3 2 4 3 4 2 4
 3 1 3 4 2 4 3 2 3 2 3 4 1 4 3 4 2 2 3 3 4 4 2 4 2 4 2 3 1 4 1 1 2 4 2 2 4
 4 3 3 4 2 2 2 1 4 3 1 3 3 4 3 2 1 4 3 3 1 4 3 3 4 1 3 4 2 1 3 1 2 3 2 2 2
 3 4 1 2 1 1 4 1 2 2 4 2 4 4 4 1 4 3 1 3 1 4 1 3 1 4 2 1 2 2 1 2 2 1 1 3 2
 1 2 3 1 2 4 1 3 4 4 1 3 2 1 2 4 4 1 1 3 4 3 4 2 4 4 2 1 2 1 2 3 2 3 3 1 2
 2 3 2 2 3 1 1 1 4 3 1 1 3 1 1 1 2 4 2 4 4 1 1 1 1 2 3 1 4 4 3 2 2 3 1 3 2
 4 1 1 2 1 1 1 1 1 1 4 2 4 3 4 4 3 4 1 1 2 4 4 2 3 4 4 2 4 1 4 2 4 3 4 2 4
 3 3 1 2 3 1 2 2 1 4 2 4 1 3 1 1 2 3 2 4 4 3 4 3 4 2 2 2 2 3 4 2 2 4 4 2 2
 4 3 1 2 1 1 2 4 2 3 4 1 4 3 2 4 4 1 4 3 4 2 1 2 2 1 2 3 4 1 4 1 2 4 1 4 1
 1 2 1 4 1 2 4 3 3 3 1 1 3 4 2 4 2 2 4 2 4 3 1 3 3 4 2 2 2 2 2 4 3 4 1 1 2
 4 1 3 4 2 1 3 2 4 2

In [2]:
task_time = np.array(list(zip(difficult, timeh)))
task_time

array([[2. , 4.5],
       [3. , 9. ],
       [1. , 8.5],
       ...,
       [3. , 5. ],
       [2. , 6. ],
       [1. , 2.5]])

In [21]:
class Genetic:
    
    def __init__(self, len_chromosome, count_indiv, gen_low, gen_high,
                 employee_params, task_diff, task_time) -> None:
        self._individs = np.random.randint(low=gen_low, high=gen_high, size=[count_indiv, len_chromosome], dtype=int)
        self._params = employee_params
        self._diff = task_diff-1
        self._time = task_time
        self.len = len_chromosome
    
    def get_individs(self) -> np.ndarray:
        return self._individs
    
    def score(self, individs: np.ndarray) -> np.ndarray:
        individ_scores = np.array([])
        
        if len(individs.shape) != 1:
        
            for ind in individs:
                indiv_params_full = self._params[ind]
                ind_params = indiv_params_full[np.arange(0, self.len), self._diff]
                total_indiv_val = ind_params * self._time
                indiv_val = np.bincount(ind, weights=total_indiv_val)
                individ_scores = np.append(individ_scores, 1_000_000 / np.max(indiv_val))
        else:
            indiv_params_full = self._params[individs]
            ind_params = indiv_params_full[np.arange(0, self.len), self._diff]
            total_indiv_val = ind_params * self._time
            indiv_val = np.bincount(individs, weights=total_indiv_val)
            individ_scores = np.append(individ_scores, 1_000_000 / np.max(indiv_val))
            
        return individ_scores
    
    def selection(self, fitness: np.ndarray, n: int) -> list:
        
        total_fitness = np.sum(fitness)
        dist = total_fitness/n
        
        start = np.random.ranf() * dist
        
        pointers = [start + i*dist for i in range(n)]
        fit_sums = [np.sum(fitness[:i]) for i in range(1, len(fitness)+1)]
            
        keep = list()

        search_idx = 0
        for p in pointers:
            i = search_idx
            while fit_sums[i] < p:
                i += 1
            keep.append(i)
            search_idx = i
                
        return keep
    
    def evolution_k_point(self, a: np.ndarray, b: np.ndarray, points: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    
        A, B = np.copy(a), np.copy(b)
        
        full_points = np.insert([0, len(a)], 1, points)
        pair_points = list(zip(full_points[:-1], full_points[1:]))

        for i, val in enumerate(pair_points):
            if i % 2:
                A[ val[0]+1:val[1] ], B[ val[0]+1:val[1] ] = b[ val[0]+1:val[1] ], a[ val[0]+1:val[1] ]
                
        return (A,B)
    
    def solution(self, generations: int=100):
        
        best_score = 0
        
        for i in range(generations):
            
            curr_score = self.score(self._individs)    
            count_individs = len(self._individs) 
            
            curr_best_score = curr_score.max()
            if curr_best_score > best_score:
                best_score = curr_best_score
                best_individ = self._individs[np.argmax(curr_score)]
                        
            if count_individs <= 2 or best_score > 1650: 
                break
            
            #shuffle individs
            if i%50 == 0:
                
                print(f"generation: {i} \
                best score:{round(best_score/10_000,4)}, \
                curr best score: {round(curr_best_score/10_000,4)}, \
                count individs: {count_individs}")
                
                random_index = np.arange(0, count_individs)
                np.random.shuffle(random_index)
                self._individs = self._individs[random_index]
                
                idx = np.random.randint(0, count_individs, int(count_individs/10))
                for i in idx:
                    np.random.shuffle(self._individs[i])
            
            self._individs = self._individs[self.selection(curr_score, int(count_individs/2))]

            all_parents = list(zip(self._individs[:-1], self._individs[1:]))
            
            for parent in all_parents[:int(count_individs/4)]:
                self._individs = np.vstack([    self._individs, 
                                                self.evolution_k_point(parent[0], parent[1], np.random.randint(2, int(self.len/2)))])
            
        return best_individ
               
            
            
            
        

In [22]:
genetic = Genetic(count_tasks, 250, 0, len(params), params, difficult, timeh)
best = genetic.solution(20000)
score = genetic.score(best)

output = open("output.txt", "w")
for i in best:
    output.write(str(i+1)+' ')
output.close()

print(f"best score:{score}\n")#best individ:\n{best+1}")

generation: 0                     best score:0.1441,                     curr best score: 0.1441,                     count individs: 250
generation: 50                     best score:0.1518,                     curr best score: 0.145,                     count individs: 248
generation: 100                     best score:0.1518,                     curr best score: 0.1462,                     count individs: 248
generation: 150                     best score:0.1532,                     curr best score: 0.1532,                     count individs: 248
generation: 200                     best score:0.1542,                     curr best score: 0.1498,                     count individs: 248
generation: 250                     best score:0.1542,                     curr best score: 0.1512,                     count individs: 248


KeyboardInterrupt: 